[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/umsynbio/TransferLearning/blob/main/transfer_learning.ipynb)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

In [10]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.mps.is_available() else 'cpu')

print(device)


mps


In [11]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG19 expects 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [12]:

# CIFAR-10 Dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)


In [13]:
# pretrained vgg 19
vgg19 = models.vgg19(weights= models.VGG19_Weights.IMAGENET1K_V1)


In [14]:
for param in vgg19.features.parameters():
    param.requires_grad = False

In [15]:
# Replace the final classifier layer for 10 CIFAR classes
vgg19.classifier[6] = nn.Linear(in_features=4096, out_features=10)
vgg19 = vgg19.to(device)

In [16]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg19.parameters(), lr=1e-4)


In [19]:
import wandb

# Initialize wandb
wandb.init(project="vgg19-cifar10")

# Watch the model to log gradients and parameters
wandb.watch(vgg19, log="all")


In [20]:

epochs = 4

# Training loop
vgg19.train()
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = vgg19(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")

    # Log to wandb
    wandb.log({"epoch": epoch+1, "loss": epoch_loss})

# Save the model weights
model_path = 'vgg19_cifar10.pth'
torch.save(vgg19.state_dict(), model_path)
print("Training complete. Model saved.")

# log the model file
wandb.save(model_path)

# Finish wandb run
# wandb.finish()

KeyboardInterrupt: 

In [ ]:
validation_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
validation_loader = DataLoader(validation_dataset, batch_size=128, shuffle=False)


vgg19.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in validation_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = vgg19(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Validation Accuracy: {accuracy:.2f}%")

try:
  wandb.log({"validation_accuracy": accuracy})
except:
  print("Wandb is not initialized. Cannot log validation accuracy.")
